In [2]:
import re
import numpy as np
import pandas as pd 

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from keras.datasets import imdb

from keras.utils.np_utils import to_categorical

import warnings
warnings.filterwarnings('ignore')
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

C:\Users\VIDUSHI\anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
max_features = 1000
maxlen = 80  # cut texts after this number of words (among top max_features most common words)
batch_size = 32

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

Loading data...
25000 train sequences
25000 test sequences


In [4]:
print(x_test.shape)
print(x_train.shape)

(25000,)
(25000,)


In [5]:
print(x_test)
print(x_train)#these are the numbers which means no. corresponding to the corpus

[list([1, 591, 202, 14, 31, 6, 717, 10, 10, 2, 2, 5, 4, 360, 7, 4, 177, 2, 394, 354, 4, 123, 9, 2, 2, 2, 10, 10, 13, 92, 124, 89, 488, 2, 100, 28, 2, 14, 31, 23, 27, 2, 29, 220, 468, 8, 124, 14, 286, 170, 8, 157, 46, 5, 27, 239, 16, 179, 2, 38, 32, 25, 2, 451, 202, 14, 6, 717])
 list([1, 14, 22, 2, 6, 176, 7, 2, 88, 12, 2, 23, 2, 5, 109, 943, 4, 114, 9, 55, 606, 5, 111, 7, 4, 139, 193, 273, 23, 4, 172, 270, 11, 2, 2, 4, 2, 2, 109, 2, 21, 4, 22, 2, 8, 6, 2, 2, 10, 10, 4, 105, 987, 35, 841, 2, 19, 861, 2, 5, 2, 2, 45, 55, 221, 15, 670, 2, 526, 14, 2, 4, 405, 5, 2, 7, 27, 85, 108, 131, 4, 2, 2, 2, 405, 9, 2, 133, 5, 50, 13, 104, 51, 66, 166, 14, 22, 157, 9, 4, 530, 239, 34, 2, 2, 45, 407, 31, 7, 41, 2, 105, 21, 59, 299, 12, 38, 950, 5, 2, 15, 45, 629, 488, 2, 127, 6, 52, 292, 17, 4, 2, 185, 132, 2, 2, 2, 488, 2, 47, 6, 392, 173, 4, 2, 2, 270, 2, 4, 2, 7, 4, 65, 55, 73, 11, 346, 14, 20, 9, 6, 976, 2, 7, 2, 861, 2, 5, 2, 30, 2, 2, 56, 4, 841, 5, 990, 692, 8, 4, 2, 398, 229, 10, 10, 13, 2, 6

In [36]:
print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print('y train',y_train)
print('y_test',y_test)

Pad sequences (samples x time)
x_train shape: (25000, 80)
x_test shape: (25000, 80)
y train [1 0 0 ... 0 1 0]
y_test [0 1 1 ... 0 0 0]


In [12]:
print(x_train)
print(x_test)#the length of corpus is decreased only most frequent words are there

[[ 15 256   4 ...  19 178  32]
 [125  68   2 ...  16 145  95]
 [645 662   8 ...   7 129 113]
 ...
 [529 443   2 ...   4   2   2]
 [286   2  23 ...  12   9  23]
 [ 97  90  35 ... 204 131   9]]
[[  0   0   0 ...  14   6 717]
 [  2 398 229 ... 125   4   2]
 [687   2 203 ...   9  57 975]
 ...
 [  0   0   0 ...  21 846   2]
 [  8  97  14 ...   2   7 470]
 [718   2   9 ...  34   2   2]]


In [194]:
INDEX_FROM=3
word_to_id=imdb.get_word_index()
#print(word_to_id)
word_to_id={k:(v+INDEX_FROM)for k,v in word_to_id.items()}
#print(word_to_id)

In [24]:
word_to_id["<PAD>"]=0
word_to_id["<START>"]=1
word_to_id["<UNK>"]=2

id_to_word={value:key for key,value in word_to_id.items()}
print(' '.join(id_to_word[id] for id in x_train[0]  ))#####################doubt

that played the <UNK> of <UNK> and paul they were just brilliant children are often left out of the <UNK> <UNK> i think because the stars that play them all <UNK> up are such a big <UNK> for the whole film but these children are amazing and should be <UNK> for what they have done don't you think the whole story was so <UNK> because it was true and was <UNK> life after all that was <UNK> with us all


In [40]:
print('Build model.....')
model=Sequential()
model.add(Embedding(max_features,32))
model.add(LSTM(128,dropout=0.2,recurrent_dropout=0.2))
model.add(Dense(1,activation='sigmoid'))# 1 is the no of neurons in sigmoid layer
model.summary()

Build model.....
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, None, 32)          32000     
_________________________________________________________________
lstm_4 (LSTM)                (None, 128)               82432     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 129       
Total params: 114,561
Trainable params: 114,561
Non-trainable params: 0
_________________________________________________________________


In [42]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print('Train')
model.fit(x_train,y_train,batch_size=batch_size,epochs=1,validation_data=(x_test,y_test))

Train
Train on 25000 samples, validate on 25000 samples
Epoch 1/1
25000/25000 [==============================] - 116s 5ms/step - loss: 0.4391 - acc: 0.8009 - val_loss: 0.4210 - val_acc: 0.8054


In [27]:
score,acc=model.evaluate(x_test,y_test,batch_size=batch_size)
print("Test score:",score)
print("accuracy",acc)

25000/25000 [==============================] - 22s 897us/step
Test score: 0.4314479623031616
accuracy 0.80368


In [48]:
prediction=model.predict(x_test[0:1])
print('Prediction value',prediction[0])
print('Test label',y_test[2:3])

Prediction value [0.4917033]
Test label [1]


In [46]:
model=Sequential()
model.add(Embedding(max_features,32))
model.add(LSTM(8,dropout=0.9,recurrent_dropout=0.9))
model.add(Dense(1,activation='sigmoid'))


In [47]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(x_train,y_train,batch_size=batch_size,epochs=1,validation_data=(x_test,y_test))
score,acc=model.evaluate(x_test,y_test,batch_size)
print('Test score',score)
print('Test accuracy',acc)

Train on 25000 samples, validate on 25000 samples
Epoch 1/1
25000/25000 [==============================] - 9s 375us/step
Test score 0.6668669400787354
Test accuracy 0.63948


In [52]:
predict=model.predict(x_test[5:6])
print('prediction value',prediction[0])
print('Test label',y_test[5:6])

prediction value [0.4917033]
Test label [1]


In [253]:
#lets test it with the custom data
import pandas as pd
data=pd.read_csv('senti.csv')
data=data[['text','sentiment']]
data.head(15)

,text,sentiment
0,I love this car,Positive
1,This view is amazing,Positive
2,I feel great this morning,Positive
3,I am so excited about the concert,Positive
4,He is my best friend,Positive
5,I do not like this car,Negative
6,This view is horrible,Negative
7,I feel tired this morning,Negative
8,I am not looking forward to the concert,Negative


In [276]:
#format the data
data=data[data.sentiment!='Neutral']
data['text']=data['text'].apply(lambda x:x.lower())
data['text']=data['text'].apply((lambda x:re.sub('[^a-zA-Z0-9\s]','',x)))#it replaces all the remaining characters except the range given
print(data)

                                     text sentiment
0                         i love this car  Positive
1                    this view is amazing  Positive
2               i feel great this morning  Positive
3        i am so excited about the conce   Positive
4                    he is my best friend  Positive
5                  i do not like this car  Negative
6                   this view is horrible  Negative
7               i feel tired this morning  Negative
8  i am not looking forward to the conce   Negative


In [277]:
print('positive samples',data[data['sentiment']=='Positive'].size)

positive samples 10


In [278]:
print('Negative samples',data[data['sentiment']=='Negative'].size)
print(row[0])

Negative samples 8
i am not looking forward to the conce 


In [279]:
for idx,row in data.iterrows(): #iterate row wise
    row[0]=row[0].replace('rt',' ')


In [280]:
print(row[0])

i am not looking forward to the conce 


In [281]:
max_fatures=2000

In [282]:
tokenizer=Tokenizer(nb_words=max_fatures,split=' ')
tokenizer.fit_on_texts(data['text'].values)
print(data['text'].values)
print(tokenizer.fit_on_texts)

['i love this car' 'this view is amazing' 'i feel great this morning'
 'i am so excited about the conce ' 'he is my best friend'
 'i do not like this car' 'this view is horrible'
 'i feel tired this morning' 'i am not looking forward to the conce ']
<bound method Tokenizer.fit_on_texts of <keras_preprocessing.text.Tokenizer object at 0x000001DE47FCE8D0>>


In [283]:
X=tokenizer.texts_to_sequences(data['text'].values)
print(X)
X=pad_sequences(X)#pads the sequences
print(X)

[[1, 12, 2, 4], [2, 5, 3, 13], [1, 6, 14, 2, 7], [1, 8, 15, 16, 17, 9, 10], [18, 3, 19, 20, 21], [1, 22, 11, 23, 2, 4], [2, 5, 3, 24], [1, 6, 25, 2, 7], [1, 8, 11, 26, 27, 28, 9, 10]]
[[ 0  0  0  0  1 12  2  4]
 [ 0  0  0  0  2  5  3 13]
 [ 0  0  0  1  6 14  2  7]
 [ 0  1  8 15 16 17  9 10]
 [ 0  0  0 18  3 19 20 21]
 [ 0  0  1 22 11 23  2  4]
 [ 0  0  0  0  2  5  3 24]
 [ 0  0  0  1  6 25  2  7]
 [ 1  8 11 26 27 28  9 10]]


In [284]:
Y=pd.get_dummies(data['sentiment']).values#split the sentiment into 1,0 see the function of get_dummies
print(Y)#[0,1]-positive
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.33,random_state=42)

[[0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]]


In [285]:
print(X_train)
print(Y_train)
print(X_test)

[[ 0  0  0  0  1 12  2  4]
 [ 1  8 11 26 27 28  9 10]
 [ 0  0  0  1  6 14  2  7]
 [ 0  0  0 18  3 19 20 21]
 [ 0  1  8 15 16 17  9 10]
 [ 0  0  0  0  2  5  3 24]]
[[0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [1 0]]
[[ 0  0  0  1  6 25  2  7]
 [ 0  0  0  0  2  5  3 13]
 [ 0  0  1 22 11 23  2  4]]


In [286]:
model=Sequential()
model.add(Embedding(max_features,128,input_length=X.shape[1],dropout=0.2))
model.add(LSTM(128))
model.add(Dense(2,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_18 (Embedding)     (None, 8, 128)            256000    
_________________________________________________________________
lstm_17 (LSTM)               (None, 128)               131584    
_________________________________________________________________
dense_17 (Dense)             (None, 2)                 258       
Total params: 387,842
Trainable params: 387,842
Non-trainable params: 0
_________________________________________________________________
None


In [287]:
batch_size=32
model.fit(X_train,Y_train,epochs=5,batch_size=batch_size,verbose=2)

Epoch 1/5
 - 2s - loss: 0.6928 - acc: 0.5000
Epoch 2/5
 - 0s - loss: 0.6813 - acc: 0.6667
Epoch 3/5
 - 0s - loss: 0.6714 - acc: 0.6667
Epoch 4/5
 - 0s - loss: 0.6593 - acc: 0.6667
Epoch 5/5
 - 0s - loss: 0.6462 - acc: 0.6667


In [288]:
score,acc=model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size)
print("Score: %.3f" %score)#loss
print("Accuracy: %.2f" %(acc))
prediction=model.predict(X_test[1:2])
print(prediction[0])
print('test label',Y_test[1:2])

Score: 0.745
Accuracy: 0.33
[0.45576233 0.5442377 ]
test label [[0 1]]


In [289]:
text='he is good boy'
tester=np.array([text])
print(tester)
tester=pd.DataFrame(tester)#convert into a table with rows and columns
print(tester)
tester.columns=['text']#write the column as 'text'
print(tester)

['he is good boy']
                0
0  he is good boy
             text
0  he is good boy


In [290]:
tester['text']=tester['text'].apply(lambda x:x.lower())
#tester['text']=tester['text'].apply((lambda x:re.sub('[^a-zA-z0-9\s]','',x)
print(tester)

             text
0  he is good boy


In [291]:
#formatting (pre-processing)
#1.convert ur data to tabular format then to lower case,then use re.sub by replacing the special characters
#2.read rowwise,by iterrows
#3.throgh tokenizer classes ,tokenize,split by' ',convert ur words into (random)no's
#then pad those no by 0-----------this was all for X
#now using get_dummies do this for y ,
#randomly segregate the test and train set

In [292]:
max_fatures = 2000
tokenizer = Tokenizer(nb_words=max_fatures, split=' ')
tokenizer.fit_on_texts(tester['text'].values)
test = tokenizer.texts_to_sequences(tester['text'].values)
print(test)
test = pad_sequences(test)
print(test)

if X.shape[1]>test.shape[1]:
    test = np.pad(test[0], (X.shape[1]-test.shape[1],0), 'constant')
    
test = np.array([test])
print(test)# without converting it into array it will give error
prediction = model.predict(test)
print('Prediction value:',prediction[0])

[[1, 2, 3, 4]]
[[1 2 3 4]]
[[0 0 0 0 1 2 3 4]]
Prediction value: [0.43992394 0.560076  ]
